In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from utils import DATABASE_DIR
# =====================================================
# Leitura do dataset
# =====================================================
df = pd.read_csv( DATABASE_DIR / 'Comun_Ocorrencias_final.csv', 
                 on_bad_lines='skip',
                 encoding='utf-8',
                 sep=',',
                 header=0)

# Remoção de colunas irrelevantes
cols_drop = [
    'Data', 'Relato da Ocorrência', 'Ações Imediatas', 
    'Ação Posterior/Programada', 'Criado', 'Criado por', 
    'QTD Plano de Ação'
]
df = df.drop(columns=[c for c in cols_drop if c in df.columns])

# =====================================================
# Seleção de amostra "Reportável"
# =====================================================
#df = df[df["Situação Reporte"] == "Reportável"]

# Define a coluna alvo
target_col = "Situação Reporte"
targetIndex = df.columns.get_loc(target_col)

X = df.drop(columns=[target_col])
y = df[target_col].values

# =====================================================
# Codificação One-Hot para as features categóricas
# =====================================================
onehotencoder_X = ColumnTransformer(
    transformers=[
        ("OneHot", OneHotEncoder(handle_unknown='ignore'), list(range(X.shape[1])))
    ], 
    remainder='passthrough'
)

X = onehotencoder_X.fit_transform(X)

# =====================================================
# Codificação do alvo
# =====================================================
onehotencoder_y = OneHotEncoder(sparse_output=False)
y = onehotencoder_y.fit_transform(y.reshape(-1, 1))

# =====================================================
# Divisão treino/teste
# =====================================================
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# =====================================================
# Definição da rede neural
# =====================================================
def nn_create():
    model = Sequential()
    model.add(Dense(units=356, activation='relu', kernel_initializer='random_uniform', input_dim=X_treino.shape[1]))
    model.add(Dropout(0.1))
    model.add(Dense(units=356, activation='relu', kernel_initializer='random_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(units=y.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = nn_create()
model.summary()

# =====================================================
# Treinamento
# =====================================================
history = model.fit(
    X_treino, y_treino,
    validation_split=0.2,
    epochs=50,
    batch_size=100,
    verbose=1
)

# =====================================================
# Avaliação
# =====================================================
loss, acc = model.evaluate(X_teste, y_teste, verbose=0)
print(f'\nAcurácia no conjunto de teste: {acc:.4f}')

# =====================================================
# Gráficos de desempenho
# =====================================================

# --- Gráfico de acurácia ---
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.show()

# --- Gráfico de perda ---
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda (Loss) do Modelo')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# =====================================================
# Previsões e matriz de confusão
# =====================================================
y_pred = model.predict(X_teste)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_teste, axis=1)

cm = confusion_matrix(y_true_classes, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("Matriz de Confusão - Teste")
plt.show()

# Salvar em formato HDF5 (arquivo único .h5)
model.save('../modelos/modelo_situacao_reporte_new.keras')
print("Modelo salvo em 'modelo_situacao_reporte_new.keras' com sucesso!")
